<a href="https://colab.research.google.com/github/kavyajeetbora/monitoring_water_surface_area/blob/master/notebooks/Plot%20the%203D%20terrain%20from%20DEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimating the lake depth


In [1]:
!pip install -q rioxarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 32.4 MB/s eta 0:00:00


In [23]:
import geemap
import ee
import matplotlib.pyplot as plt
import pandas as pd
import rioxarray as rxr
import numpy as np
import plotly.graph_objects as go
import scipy as sp
import matplotlib

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Area of Interest

Used this tool to create a geojson file of the area of interest:

[keene Polyline Tool](https://www.keene.edu/campus/maps/tool/?coordinates=77.1200409%2C%2011.5324541%0A76.9923248%2C%2011.5062217%0A76.9916382%2C%2011.3467571%0A77.1529998%2C%2011.4261642%0A77.1200409%2C%2011.5324541) : Use this tool to create a polygon and generate a GeoJson text for further use

In [41]:
geojson  = {
  "coordinates": [
    [
      [
        91.6496658,
        26.1440994
      ],
      [
        91.6496658,
        26.1440994
      ],
      [
        91.6315985,
        26.1416338
      ],
      [
        91.6183376,
        26.1313856
      ],
      [
        91.6145611,
        26.1183236
      ],
      [
        91.6118574,
        26.1035645
      ],
      [
        91.6227293,
        26.0965504
      ],
      [
        91.6407967,
        26.0999419
      ],
      [
        91.6599369,
        26.104027
      ],
      [
        91.6800642,
        26.1113875
      ],
      [
        91.6816664,
        26.1219714
      ],
      [
        91.6752148,
        26.1445361
      ],
      [
        91.6496658,
        26.1440994
      ]
    ]
  ],
  "type": "Polygon"
}

geometry = ee.Geometry(geojson)

## Downloading the Global lakes bathymetry dataset

[Global lakes bathymetry dataset](https://gee-community-catalog.org/projects/globathy/): bathymetric data of 1.4+ million waterbodies to align with the well-established global dataset, HydroLAKES. GLOBathy uses a GIS-based framework to generate bathymetric maps based on the waterbody maximum depth estimates and HydroLAKES geometric/geophysical attributes of the waterbodies. The maximum depth estimates are validated at 1,503 waterbodies, making use of several observed data sources

In [42]:
globathy = ee.Image("projects/sat-io/open-datasets/GLOBathy/GLOBathy_bathymetry")
globathy = globathy.multiply(-1).rename('DSM').unmask(0) ## Multiplying -1 to represent the data in negative

Map = geemap.Map(basemap='OpenTopoMap')
visParams = {"min": -20, "max": 0, 'palette': ['#d73027','#f46d43','#fdae61','#fee090','#e0f3f8','#abd9e9','#74add1','#4575b4']}
Map.addLayer(globathy.clip(geometry), visParams, 'Global Bathymetry')
Map.centerObject(geometry, zoom=12)
Map

Map(center=[26.121530947020897, 91.64679638900466], controls=(WidgetControl(options=['position', 'transparent_…

## Load the elevation model

[All you need to know about digital elevation models](https://up42.com/blog/everything-you-need-to-know-about-digital-elevation-models-dem-digital)

- Load the elevation model
- add the lake bathymetry depths to the elevation model to get the lake depressions

In [43]:
alos_dsm = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2")\
.filter(ee.Filter.bounds(geometry))

alos_image = alos_dsm.mosaic().select("DSM")

## Add the two images
terrain = alos_image.add(globathy)

In [44]:
Map = geemap.Map(basemap='OpenTopoMap')
palette = ['#543005','#8c510a','#bf812d','#dfc27d','#f6e8c3','#c7eae5','#80cdc1','#35978f','#01665e','#003c30']
visParams = {"bands": ['DSM'], "min": 0, "max": 500, 'palette': palette}

Map.addLayer(alos_image.clip(geometry), visParams, 'ALOS Global DSM-30m')
Map.addLayer(geometry, {}, "boundary")
Map.centerObject(geometry, zoom=12)
Map

Map(center=[26.121530947020897, 91.64679638900466], controls=(WidgetControl(options=['position', 'transparent_…

In [45]:
spatial_resolution = 30
geemap.ee_export_image(
    terrain, filename="terrain.tif", scale=spatial_resolution, region=geometry, file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to /content/terrain.tif


In [46]:
def matplotlib_to_plotly(cmap, pl_entries):
    '''Converts a matplolib colorscale to plotly colorscale'''
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

def smoothen_dataArray(tif_file, sigma = 5):
    '''
    This function reads an tif image containing the terrain data
    Smoothens the elevation data using gaussian filter based on sigma value
    Input: .tif file
    returns: a processed Xarray.Dataset containing the terrain data
    '''
    da = rxr.open_rasterio(filename=tif_file)
    da_vals = da.isel(band=0).values ## Select the first band of the image
    # Apply gaussian filter, with sigmas as variables. Higher sigma = more smoothing and more calculations. Downside: min and max values do change due to smoothing
    sigma = [sigma, sigma] ## Sigma values in x and y direction
    z_smoothed = sp.ndimage.gaussian_filter(da_vals, sigma)

    da.data = np.expand_dims(z_smoothed, axis=0)
    return da


def plot_3D_terrain(xr_array, total_vol, colorscale = 'Earth', spatial_resolution=30, depth_scale=1):

    '''
    Plots an array data in 3D.
    Input: Xarray.DataArray
    Returns: plotly figure representing the 3D terrain data
    '''

    Z = xr_array.values ## Z are the elevation/depth values
    Y = xr_array['y'].values ## Y - latitude values
    X = xr_array['x'].values ## X - longitude values
    x_ratio, y_ratio = xr_array.shape

    fig = go.Figure()

    fig.add_trace(
        go.Surface(
            z=Z,
            x=X,
            y=Y,
            colorscale = colorscale,
            hovertemplate ='Depth: %{z:.2f} m',
            name=""
        )
    )

    fig.update_layout(
        margin=dict(l=0, r=0, t=30, b=0),
        title = f"Lake Topography | Total Estimated Volume: {total_vol} ML",
        plot_bgcolor="rgba(0, 0, 0, 0)",   # Transparent plot background
    )

    # Set the box aspect ratio (equal scales for all axes)
    fig.update_scenes(
        aspectratio=dict(x=spatial_resolution, y=spatial_resolution,z=depth_scale),
        xaxis=dict(showticklabels=False, title="", showgrid=False),
        yaxis=dict(showticklabels=False, title="", showgrid=False),
        zaxis=dict(showticklabels=False, title="Depth (m)→", showgrid=False)
    )

    return fig

In [47]:
## Convert colorscales from matplotlib to plotly colorscales
terrain_cmap = matplotlib.cm.get_cmap('winter')
terrain_colorscale = matplotlib_to_plotly(terrain_cmap, 255)

## Read and smoothen the DEM tif image file
ds = smoothen_dataArray('terrain.tif', sigma=5)
da = ds.isel(band=0) ## Select the first array

fig = plot_3D_terrain(da, "", colorscale=terrain_colorscale)
fig.show()